In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
%matplotlib inline

In [ ]:
building_df = pd.read_csv("../input/ashrae-energy-prediction/building_metadata.csv")
weather_train = pd.read_csv("../input/ashrae-energy-prediction/weather_train.csv")
train = pd.read_csv("../input/ashrae-energy-prediction/train.csv")

In [ ]:
train = train.merge(building_df, left_on = "building_id", right_on = "building_id", how = "left")
train = train.merge(weather_train, left_on = ["site_id", "timestamp"], right_on = ["site_id", "timestamp"], how = "left")
del weather_train

In [ ]:
# Обнаружили незаполненный target до 20.05.2016
# train[(train.meter_reading==0) & (train.meter==0) & (train.site_id==0)].groupby('timestamp').building_id.count() 

In [ ]:
train = train.query('not (building_id <= 104 & meter == 0 & timestamp <= "2016-05-20 17:00:00")')

In [ ]:
train['wind_direction'] = train['wind_direction'].map(lambda x: 
                                                    1 if x <= 22.5 and x >= 337.5
                                                    else 2 if x > 22.5 and x < 67.5
                                                    else 3 if x >= 67.5 and x <= 112.5
                                                    else 4 if x > 112.5 and x < 157.5
                                                    else 5 if x >= 157.5 and x <= 202.5
                                                    else 6 if x > 202.5 and x < 247.5
                                                    else 7 if x >= 247.5 and x <= 292.5                  
                                                    else 8)

In [ ]:
train.info()

In [ ]:
train['timestamp'] = pd.to_datetime(train.timestamp)

train['hour'] = train['timestamp'].dt.hour
train['weekend'] = train['timestamp'].dt.weekday
train['month'] = train['timestamp'].dt.month
train.drop(['timestamp'], axis=1, inplace=True)

In [ ]:
train['building_id'] = train['building_id'].astype('object')
train['meter'] = train['meter'].astype('object')
train['cloud_coverage'] = train['cloud_coverage'].astype('object')
train['wind_direction'] = train['wind_direction'].astype('object')
train['site_id'] = train['site_id'].astype('object')
train['hour'] = train['hour'].astype('object')
train['weekend'] = train['weekend'].astype('object')
train['month'] = train['month'].astype('object')

### NAN 

In [ ]:
def show_nan(df):
# Показать процент пропущенных данных по признакам

    total = df.isnull().sum().sort_values(ascending=False)
    percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
    missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    print(missing_data.head(20))

In [ ]:
show_nan(train)

In [ ]:
#Fill Nan value in weather dataframe by interpolation
train = train.groupby('site_id').apply(lambda group: group.interpolate(limit_direction='both'))

In [ ]:
train = train.drop(['year_built'], axis=1)
train['floor_count'] = train['floor_count'].fillna(1)
train['precip_depth_1_hr'] = train['precip_depth_1_hr'].fillna(0)
train['sea_level_pressure'] = train['sea_level_pressure'].fillna(train['sea_level_pressure'].median())
train['cloud_coverage'] = train['cloud_coverage'].fillna(train['cloud_coverage'].median())
train['wind_speed'] = train['wind_speed'].fillna(train['wind_speed'].median())
train['wind_direction'] = train['wind_direction'].fillna(train['wind_direction'].median())
train['dew_temperature'] = train['dew_temperature'].fillna(train['dew_temperature'].median())
train['air_temperature'] = train['air_temperature'].fillna(train['air_temperature'].median())

In [ ]:
# log transformation
train['meter_reading'] = np.log1p(train['meter_reading'])

y_train = train['meter_reading']
train.drop(['meter_reading'], axis=1, inplace=True)

In [ ]:
#Based on this great kernel https://www.kaggle.com/arjanso/reducing-dataframe-memory-size-by-65
def reduce_mem_usage(df):
    start_mem_usg = df.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    for col in df.columns:
        if df[col].dtype != object:  # Exclude strings            
            IsInt = False
            mx = df[col].max()
            mn = df[col].min()
            # test if column can be converted to an integer
            asint = df[col].fillna(0).astype(np.int64)
            result = (df[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True            
            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        df[col] = df[col].astype(np.uint8)
                    elif mx < 65535:
                        df[col] = df[col].astype(np.uint16)
                    elif mx < 4294967295:
                        df[col] = df[col].astype(np.uint32)
                    else:
                        df[col] = df[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)    
            # Make float datatypes 32 bit
            else:
                df[col] = df[col].astype(np.float32)           
            
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = df.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return df

In [ ]:
train = reduce_mem_usage(train)

In [ ]:
# Преобразование признаков с отклонениями
numerical_features = train.select_dtypes(exclude = ["object"]).columns
skewness = train[numerical_features].skew()
skewed_features = skewness[abs(skewness) > 0.5].index
train[skewed_features] = np.log1p(train[skewed_features])

In [ ]:
# Преобразование категориальных признаков в таблицу 0 и 1 созданием фиктивных колонок

train = pd.get_dummies(train)
print(train.shape)

In [ ]:
# Нормализация признаков (одинаковый масштаб)

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
train = scaler.fit_transform(train)

In [ ]:
#weather_test = pd.read_csv("weather_test.csv")
#test = pd.read_csv("test.csv")

In [ ]:
#test_ID = test['row_id']
#test.drop('row_id', axis = 1, inplace = True)

In [ ]:
#test = test.merge(building_df, left_on = "building_id", right_on = "building_id", how = "left")
#test = test.merge(weather_test, left_on = ["site_id", "timestamp"], right_on = ["site_id", "timestamp"], how = "left")
#del weather_test, building_df

In [ ]:
#test = reduce_mem_usage(test)